In [3]:
from __future__ import print_function
import torch
from torchvision import datasets, transforms
import numpy as np
from WeakLearners import BoostedWongNeuralNet
import matplotlib.pyplot as plt

In [4]:
from Boosting import Ensemble, runBoosting

In [5]:
import utils

In [6]:
cuda = torch.device('cuda:0')

In [7]:
%load_ext autoreload
%autoreload 2

## MNIST Boosting

In [8]:
maxIt_vals = [10]

In [9]:
num_wl = 50 # maybe around 100? #later: maybe change this to an array?
num_epochs = 1

In [10]:
all_val_accuracies = []
all_train_accuracies_ensemble = []
all_wl = []
all_wl_weights = []

In [ ]:
for maxIt in maxIt_vals:
        wl, wl_weights, val_accuracies, train_accuracies_ensemble = runBoosting(num_wl, maxIt, num_epochs, dataset=datasets.MNIST, weakLearnerType = BoostedWongNeuralNet)
        all_val_accuracies.append(val_accuracies)
        all_wl.append(wl)
        all_wl_weights.append(wl_weights)
        all_train_accuracies_ensemble.append(train_accuracies_ensemble)

----------------------------------------------------------------------------------------------------
Training 0th weak learning
Epoch: 0


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:88: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img, target = self.data[index], int(self.targets[index])


WL has validation accuracy (1.0841253995895386, 0.73)
targets: tensor([5, 0, 4,  ..., 5, 6, 8])
Full Accuracy:  0.7667666666666667
Alpha:  0.9523649718482494
After newest WL validation score is:  0.78
After newest WL training score is:  0.775
----------------------------------------------------------------------------------------------------
Training 1th weak learning
Epoch: 0
WL has validation accuracy (0.9353989958763123, 0.65)
targets: tensor([5, 0, 4,  ..., 5, 6, 8])
Full Accuracy:  0.66245
Alpha:  0.5292656425862772
After newest WL validation score is:  0.7625000000000001
After newest WL training score is:  0.7825
----------------------------------------------------------------------------------------------------
Training 2th weak learning
Epoch: 0
WL has validation accuracy (1.0712180137634277, 0.6)
targets: tensor([5, 0, 4,  ..., 5, 6, 8])
Full Accuracy:  0.64685
Alpha:  0.5673218419033678
After newest WL validation score is:  0.7525
After newest WL training score is:  0.7525
--

In [ ]:
legend_labels = []
for i in range(len(maxIt_vals)):
    maxIt = maxIt_vals[i]
    val_accuracies = all_val_accuracies[i]
    num_wl = len(all_wl[i])
    plt.plot(np.arange(maxIt, num_wl * maxIt + 1, maxIt), val_accuracies)
    legend_labels.append(f"MaxIt: {maxIt}")
plt.legend(legend_labels)
plt.show()
    